In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload


# Basic imports
import hvplot.xarray
import numpy as np
import xarray as xr
import xclim.indices.stats as stats  # not in original example
import xdatasets as xd
from scipy.stats import genextreme, genpareto, gumbel_r  # not in original example

import xhydro_temp as xh
import xhydro_temp.frequency_analysis as xhfa
from xhydro_temp.extreme_value_analysis.parameterestimation import *

In [ ]:
import xhydro_temp.extreme_value_analysis.parameterestimation

### Dataset Setup

In [ ]:
ds = (
    xd.Query(
        **{
            "datasets": {
                "deh": {
                    "id": ["020*"],
                    "regulated": ["Natural"],
                    "variables": ["streamflow"],
                }
            },
            "time": {"start": "1970-01-01", "minimum_duration": (15 * 365, "d")},
        }
    )
    .data.squeeze()
    .load()
)

# This dataset lacks some of the aforementioned attributes, so we need to add them.
ds["id"].attrs["cf_role"] = "timeseries_id"
ds["streamflow"].attrs = {
    "long_name": "Streamflow",
    "units": "m3 s-1",
    "standard_name": "water_volume_transport_in_river_channel",
    "cell_methods": "time: mean",
}

# Some examples
timeargs = {
    "spring": {"date_bounds": ["02-11", "06-19"]},
    "summer": {"doy_bounds": [152, 243]},
    "fall": {"month": [9, 10, 11]},
    "winter": {
        "season": ["DJF"],
        "freq": "YS-DEC",
    },  # To correctly wrap around the year, we need to specify the resampling frequency.
    "august": {"month": [8]},
    "annual": {},
}
# Here, we hide years with more than 15% of missing data.
ds_4fa = xh.indicators.get_yearly_op(
    ds, op="max", timeargs=timeargs, missing="pct", missing_options={"tolerance": 0.15}
)

# Create a mask beforehand
import random

nyears = np.unique(ds.time.dt.year).size
dom_start = xr.DataArray(
    np.random.randint(1, 30, size=(nyears,)).astype("str"),
    dims=("year"),
    coords={"year": np.unique(ds.time.dt.year)},
)
dom_end = xr.DataArray(
    np.random.randint(1, 30, size=(nyears,)).astype("str"),
    dims=("year"),
    coords={"year": np.unique(ds.time.dt.year)},
)

mask = xr.zeros_like(ds["streamflow"])
for y in dom_start.year.values:
    # Random mask of dates per year, between April and June.
    mask.loc[
        {
            "time": slice(
                str(y) + "-04-" + str(dom_start.sel(year=y).item()),
                str(y) + "-06-" + str(dom_end.sel(year=y).item()),
            )
        }
    ] = 1
# The name of the indexer will be used to identify the variable created here
timeargs_custom = {"custom": {}}

# We use where() to mask the data that we want to ignore
masked = ds.where(mask == 1)
# Since we masked almost all of the year, our tolerance for missing data should be changed accordingly
missing = "at_least_n"
missing_options = {"n": 45}

# We use xr.merge() to combine the results with the previous dataset.
ds_4fa = xr.merge(
    [
        ds_4fa,
        xh.indicators.get_yearly_op(
            masked,
            op="max",
            timeargs=timeargs_custom,
            missing=missing,
            missing_options=missing_options,
        ),
    ]
)
# Get a daily volume from a daily streamflow
ds["volume"] = xh.indicators.compute_volume(ds["streamflow"], out_units="hm3")

# We'll take slightly different indexers
timeargs_vol = {"spring": {"date_bounds": ["04-30", "06-15"]}, "annual": {}}

# The operation that we want here is the sum, not the max.
ds_4fa = xr.merge(
    [
        ds_4fa,
        xh.indicators.get_yearly_op(
            ds,
            op="sum",
            input_var="volume",
            timeargs=timeargs_vol,
            missing="pct",
            missing_options={"tolerance": 0.15},
            interpolate_na=True,
        ),
    ]
)

# for id in range(len(ds_4fa.coords["id"])):
#     print(ds_4fa.streamflow_max_spring[id,:].values)

print(ds_4fa.streamflow_max_summer[0, :].values)

ds_4fa

In [ ]:
import glob
import os

from xarray import open_zarr

# path_hist = glob.glob(os.path.join(r"C:\Users\KAMIL PC\Desktop\Life\Stage\Documents\non_stationary_data\eau_precpitable_CanESM5"))
path_hist = glob.glob(
    os.path.join(
        r"C:\Users\HP\Desktop\Stage\non_stationary_data\eau_precpitable_CanESM5"
    )
)

ds_hist = open_zarr(path_hist[0], consolidated=True)
ds_hist

# extreme_value_analysis.fit()

In [ ]:
params = fit(ds_4fa, dist=genextreme, method="ml")
# print(type(params.precipitable_water[:, 0, 0]))
params

# xclim.stats.fit()

In [ ]:
from lmoments3.distr import gev, gpa, gum

params = stats.fit(ds_4fa, dist=gev, method="ml")
# print(type(params.precipitable_water[:, 0, 0]))
params

### DEBUG

In [ ]:
# da = xr.DataArray(np.arange(125).reshape(5, 5, 5), dims=("x", "y", "z"))
# da

In [ ]:
# tgt_x = xr.DataArray(np.arange(0, 5), dims="points")
# tgt_y = xr.DataArray(np.arange(0, 5), dims="points")
# da2 = da.isel(x=2, y=2, z = 2)
# da2

In [ ]:
# params_extremes.precipitable_water.isel(y = 2, x = 2, dparams = 1, missing_dims = "raise")
# params_extremes.precipitable_water[2, 2, 1]

In [ ]:
# import cftime
#
# def convert_to_year_fraction(date: cftime.DatetimeNoLeap):
#     year = date.year
#     day_of_year = date.timetuple().tm_yday
#     year_fraction = day_of_year / 365
#     return year + year_fraction

In [ ]:
# time_values = [convert_to_year_fraction(date)  for date in ds_hist.time.values]
# time_values

In [ ]:
# y = list(ds_hist.precipitable_water[:,0,0].values)
# print(y)
# gevfit(y, locationcov=[Variable("time", time_values)])
# # y

In [ ]:
# data = Extremes.dataset("fremantle")
# from juliacall import convert as jl_convert
#
# fm = Extremes.gevfit(data, jl.Symbol("SeaLevel"), locationcovid=jl_convert(jl.Vector[jl.Extremes.Symbol], ["Year", "SOI"]))
#
# params = (getattr(fm, "θ̂"))
# params

In [ ]:
# y = [
#     31.4, 38.8, 63.78, 54.82, 123.2, 108.2, 108.1, 98.52,
#     42.24, 33.45, 60.0, 84.23, 48.07, 77.71, 38.37, 42.02,
#     46.0
# ]

In [ ]:
# from xhydro_temp.extreme_value_analysis.structures.conversions import *
#
# params = jl_matrix_tuple_to_py_list(Extremes.params(Extremes.gevfitbayes(py_list_to_jl_vector(y))))
# params

In [ ]:
# [sum(x) / len(params) for x in zip(*params)]

In [ ]:
import xhydro_temp.extreme_value_analysis